In [1]:
from google.cloud import pubsub_v1

import asyncio
import litellm
from litellm.integrations.custom_logger import CustomLogger
from litellm import completion, acompletion 

In [2]:
PROJECT = 'boris001'
TOPIC_ID = 'litellm'

In [3]:
import json

class PubSubLogger(CustomLogger):
    def __init__(self, topic: str, project_id = None):
        super().__init__()

        self.pubsub = pubsub_v1.PublisherClient()
        self.topic_path = self.pubsub.topic_path(project_id, topic)

        # TODO: check schema

    def sent_message(self, content):
        if isinstance(content, dict):
            data = json.dumps(content, default=str)
        else:
            data = content
        data = data.encode('utf-8')
        # TODO: handle errors
        print('Sending message')
        future = self.pubsub.publish(self.topic_path, data)
        future.result()

    
    def log_success_event(self, kwargs, response_obj, start_time, end_time): 
        self.sent_message(kwargs)

    async def async_log_success_event(self, kwargs, response_obj, start_time, end_time):
        self.sent_message(kwargs)

    async def async_pre_call_hook(self, user_api_key_dict: UserAPIKeyAuth, cache: DualCache, data: dict, call_type: Literal[
            "completion",
            "text_completion",
            "embeddings",
            "image_generation",
            "moderation",
            "audio_transcription",
        ]): 
        print(data)
        print(call_type)
        print(user_api_key_dict)

        return data 

In [4]:
customHandler = PubSubLogger(TOPIC_ID, PROJECT)

litellm.callbacks = [customHandler]

In [5]:
response = await acompletion(
    model = 'vertex_ai/gemini-1.5-flash-002',
    messages=[{ "role": "user", "content": "Hi 👋"}],
)

Sending message


In [5]:
x = customHandler.pubsub.publish(customHandler.topic_path, b'X')

In [7]:
o = x.result()

In [11]:
type(x)

google.cloud.pubsub_v1.publisher.futures.Future